<div dir="rtl">
<font face="B Nazanin" size=5>
    <b>بسم الله الرحمن الرحیم</b>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
نوت بوکی که پیش روی شماست، مراحل انجام فاز اول چالش درس هوش مصنوعی در قسمت پردازش زبان طبیعی را تعیین کرده است. در این فاز، شما با استفاده مجموعه‌ای از محتوای متن صفحات وب به همراه برچسب موضوع هر یک، مدلی را توسعه می‌دهید که توانایی دسته‌بندی متن ورودی را به یکی از 16 برچسب مجموعه‌ داده داشته باشد.
<br>
<br>
<b>چند نکته مهم</b>
<ul >
<font face="B Nazanin" size=4>
<li>
می‌توانید از کتابخانه‌های با پیاده‌سازی آماده مدل‌های پایه (مانند transformers) استفاده کنید.
</li>
<li>
توجه کنید که تمام کتابخانه های مورد نیاز را نصب کنید و از نصب شدن آن ها اطمینان حاصل کرده و نام آن ها در فایل requirements ذکر کنید.
</li>
<li>
در صورت نیاز به نصب یک کتابخانه یا دسترسی به یک فایل، مراحل نصب و دانلود آن (از یک محل عمومی یا فایل ارسال شده به همراه نوت‌بوک) می‌بایست در نوت‌بوک وجود داشته باشد.
</li>
<li>
تمامی فایل‌های مرتبط به پروژه که دارای حجم کمی هستند، باید به همراه نوت‌بوک در قالب یک فایل زیپ ارسال شوند. فایل‌هایی که حجم زیادی دارند (مانند فایل ذخیره شده یک مدل که بیشتر از ۲۰۰ مگابایت باشد) را در یک محل عمومی مانند گوگل درایو آپلود کرده و لینک دسترسی به آن را در نوت بوک قرار دهید.
</li>
</font>
</ul>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=5>
    <b>چالش درس هوش مصنوعی - پردازش زبان طبیعی</b>
    <br>
    <b>فاز اول</b>
</font>
</div>

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>بارگذاری و پیش‌پردازش داده</b>
<br>
در ابتدا می‌بایست داده‌ها را از فایل با نام dataset_phase1 بارگذاری کنید. سپس برای اینکه بتوانید مدل مناسبی را آموزش دهید،  داده‌ها را پیش پردازش کنید. این قسمت می‌تواند شامل موارد متنوعی باشد؛ از جمله حذف عبارت‌های اضافی موجود در انتهای متن‌ها، حذف یا جداسازی علائم نگارشی، توکن‌بندی کلمات، جایگزینی مواردی مثل اعداد، تاریخ‌ها و آدرس‌‌های الکترونیکی با توکن‌های خاص، حذف فاصله‌های اضافه و ... . برای این منظور، می‌توانید از کتاب‌خانه‌هایی مانند hazm هم استفاده کنید.
سپس برای اینکه بتوانید ارزیابی مناسبی از مدل خود داشته باشید، داده‌ها را به دو قسمت آموزش  (train) و ارزیابی  (validation) تقسیم کنید.
</font>
</div>

In [21]:
# Install required packages
!pip install -q transformers
!pip install -q clean-text[gpl]

In [22]:
# Import required packages

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from cleantext import clean
from tqdm.notebook import tqdm
import os
import re
import copy
import collections
from transformers import BertConfig, BertTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
from torch.optim.lr_scheduler import ExponentialLR

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!wget -O "phase_1_train.csv" 'https://drive.google.com/u/0/uc?id=1mMB1nl3Eui38w-LxbR9oDNdgEFsurSnG&export=download'

--2023-06-25 12:06:33--  https://drive.google.com/u/0/uc?id=1mMB1nl3Eui38w-LxbR9oDNdgEFsurSnG&export=download
Resolving drive.google.com (drive.google.com)... 142.251.161.139, 142.251.161.102, 142.251.161.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.161.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1mMB1nl3Eui38w-LxbR9oDNdgEFsurSnG&export=download [following]
--2023-06-25 12:06:33--  https://drive.google.com/uc?id=1mMB1nl3Eui38w-LxbR9oDNdgEFsurSnG&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-6o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/h1hvi8tv96qvcemnmddbq2qvd1n1k2nc/1687694775000/00337956063233521205/*/1mMB1nl3Eui38w-LxbR9oDNdgEFsurSnG?e=download&uuid=a2014ed9-483c-42ea-9a12-8283beba4dd0 [following]
--2023-06-25 12:06:35--  https://doc-0s-6o-docs.googleuserconten

In [25]:
data = pd.read_csv('./phase_1_train.csv', encoding='utf-8')
data = data[['text', 'class_id']]

In [26]:
def cleanhtml(raw_html):
      cleanr = re.compile('<.*?>')
      cleantext = re.sub(cleanr, '', raw_html)
      return cleantext
def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)

    # normalizing

    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [27]:
data['cleaned_comment'] = data['text'].apply(cleaning)
data = data.reset_index(drop=True)

In [28]:
data = data[['cleaned_comment', 'class_id']]
data.columns = ['comment', 'label']

In [29]:
train, test = train_test_split(data, test_size=0.2, random_state=1, stratify=data['label'])
train = train.reset_index()
test = test.reset_index()

In [30]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [31]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-zwnj-base'

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>آموزش و ارزیابی مدل</b>
<br>
در این قسمت شما به آماده‌سازی مدل انتخابی و آموزش دسته‌بند می‌پردازید. برای این‌که بتوانید به نتایج بهتری برسید، مقادیر مختلف پارامترها و ابرپارامترهای مدل را با استفاده از داده اعتبارسنجی ارزیابی کنید و بهترین مقادیر را انتخاب کنید. در ارزیابی مدل می‌توانید از معیارهای متفاوتی مانند
      Recall ،Precision ،Accuracy ، Weighted F1-Scoreو F1-Score استفاده کنید.
پس از انتخاب پارامترهای بهینه، می‌توانید مدل نهایی خود را روی تمامی داده‌ها آموزش دهید.
</font>
</div>

In [32]:
class TaaghcheDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Taaghche. """

    def __init__(self, tokenizer, comments, targets=None, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}


    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])

        if self.has_target:
            target = self.label_map[self.targets[item]]

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = TaaghcheDataset(
        comments=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        label_list=label_list)


    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [33]:
label_set = set(data['label'])
label_list = sorted(list(label_set))

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
train_data_loader = create_data_loader(train['comment'].to_numpy(), train['label'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['comment'].to_numpy(), test['label'].to_numpy(), tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [34]:
class SentimentModel(nn.Module):

    def __init__(self):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.layer = nn.Sequential(
          nn.Dropout(),
          nn.LazyLinear(64),
          nn.Dropout(),
          nn.ReLU(),
          nn.LazyLinear(16),
        )

    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)
        pooled_output = pooled_output.pooler_output
        pooled_output = self.layer(pooled_output)
        return pooled_output

In [36]:
criterion = nn.CrossEntropyLoss()
net = SentimentModel()
net = net.to(device)
optimizer = AdamW(net.parameters(), lr=2e-5, correct_bias=False)

scheduler = ExponentialLR(optimizer, gamma=0.5)

losses_train = []
losses_test = []
accuracies_test = []

for epoch in tqdm(range(10)):  # loop over the dataset multiple times
    loss_train = 0
    num_train = 0
    net.train()
    start_time = time.time()
    for dl in tqdm(train_data_loader):
        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = net(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        loss_train += loss.item() * len(outputs)
        num_train += len(outputs)

    losses_train.append(loss_train / num_train)
    elapsed_time = time.time() - start_time

    loss_test = 0
    num_test = 0

    acc_test = 0
    net.eval()
    for dl in tqdm(test_data_loader):
        input_ids = dl['input_ids'].to(device)
        attention_mask = dl['attention_mask'].to(device)
        token_type_ids = dl['token_type_ids'].to(device)
        targets = dl['targets'].to(device)

        with torch.no_grad():
          outputs = net(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

          loss = criterion(outputs, targets)
          loss_test += loss.item() * len(outputs)
          num_test += len(outputs)

          label_pred = torch.argmax(outputs, dim=1)
          acc_test += (label_pred == targets).sum().item()

    losses_test.append(loss_test / num_test)
    accuracies_test.append(100 * acc_test / num_test)
    scheduler.step()
    print(f"epoch: {epoch}, loss train : {losses_train[-1]}, loss test: {losses_test[-1]}, acc test: {accuracies_test[-1]}%, time = {elapsed_time}, lr = {optimizer.param_groups[0]['lr']}")

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

epoch: 0, loss train : 1.8077535299667877, loss test: 1.0561569881747699, acc test: 77.63401109057301%, time = 126.15375518798828, lr = 1e-05


...

  0%|          | 0/34 [00:00<?, ?it/s]

epoch: 8, loss train : 0.49692723778029496, loss test: 0.6420323040040276, acc test: 86.69131238447319%, time = 127.0745222568512, lr = 3.90625e-08


  0%|          | 0/136 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

epoch: 9, loss train : 0.5034571696850284, loss test: 0.6417663859792205, acc test: 86.69131238447319%, time = 126.74608612060547, lr = 1.953125e-08


In [37]:
state_dict = net.state_dict()
torch.save(state_dict, "/content/drive/MyDrive/NLP/checkpoint.pth")

<div dir="rtl">
<font face="B Nazanin" size=4>
<b>دسترسی به مدل</b>
<br>
برای ارزیابی مدل شما توسط داده آزمایش (test)، تابعی با نام classify_text بنویسید که به عنوان ورودی دیتافریم تست را دریافت کند و در خروجی یک لیست از اعداد فیلد 'class_id' (از جنس int) متناظر با متن دیتافریم ورودی در ردیف با همان index است را با توجه به موضوع متن تولید کند( test_dataframe.iloc[i] -> results[i] ). این تابع توسط دستیاران آموزشی روی داده‌های آزمایش اجرا شده و نتایج آن برای ارزیابی کیفیت مدل شما استفاده خواهد شد.  
</font>
</div>

In [ ]:
class ClassificationModel():
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.net = SentimentModel()
        os.system('gdown --id 1-4mDoO4-8AI_phkU6wgWUIlobcEfzf6H')
        self.net.load_state_dict(torch.load("checkpoint.pth", map_location=torch.device('cpu')))
        self.tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-fa-zwnj-base')
        self.net = self.net.to(self.device)
        self.net.eval()

    def classify_text(self, test_dataframe):
        test_dataframe = self.final_clean(test_dataframe)
        dataloader = self.create_data_loader(test_dataframe.iloc[:, 0].to_numpy(), None, self.tokenizer, 256, 16, None)



        r = []
        for dl in dataloader:
            input_ids = dl['input_ids'].to(self.device)
            attention_mask = dl['attention_mask'].to(self.device)
            token_type_ids = dl['token_type_ids'].to(self.device)
            with torch.no_grad():
                outputs = self.net(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)
                label_pred = torch.argmax(outputs, dim=1)
                for i in range(len(label_pred)):
                    val = label_pred[i].item() + 6
                    r.append(val)

        return r
    def create_data_loader(self, x, y, tokenizer, max_len, batch_size, label_list):
      dataset = TaaghcheDataset(
          comments=x,
          targets=y,
          tokenizer=tokenizer,
          max_len=max_len,
          label_list=label_list)

      return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last = False)

    def cleanhtml(self, raw_html):
        cleanr = re.compile('<.*?>')
        cleantext = re.sub(cleanr, '', raw_html)
        return cleantext
    def cleaning(self, text):
        text = text.strip()

        # regular cleaning
        text = clean(text,
            fix_unicode=True,
            to_ascii=False,
            lower=True,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=True,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=True,
            no_punct=False,
            replace_with_url="",
            replace_with_email="",
            replace_with_phone_number="",
            replace_with_number="",
            replace_with_digit="0",
            replace_with_currency_symbol="",
        )

        # cleaning htmls
        text = self.cleanhtml(text)

        # removing wierd patterns
        wierd_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            u"\U0001f926-\U0001f937"
            u'\U00010000-\U0010ffff'
            u"\u200d"
            u"\u2640-\u2642"
            u"\u2600-\u2B55"
            u"\u23cf"
            u"\u23e9"
            u"\u231a"
            u"\u3030"
            u"\ufe0f"
            u"\u2069"
            u"\u2066"
            # u"\u200c"
            u"\u2068"
            u"\u2067"
            "]+", flags=re.UNICODE)

        text = wierd_pattern.sub(r'', text)

        # removing extra spaces, hashtags
        text = re.sub("#", "", text)
        text = re.sub("\s+", " ", text)

        return text
    def final_clean(self, data):

        data['cleaned_comment'] = data.iloc[:, 0].apply(self.cleaning)
        data = data[['cleaned_comment']]
        data.columns = ['comment']

        return data

In [ ]:
https://doc-08-7o-docs.googleusercontent.com/docs/securesc/4v93l62l6v4q6gunj9sqan927fcg93jd/b73vf89an09op5oe49evuqf4lkicjkdn/1687696275000/05847473060504281705/05847473060504281705/11BDLHpof9MF2K32HOXYwpCrpywwjBYmy?e=download&ax=ADWCPKB_IsO7g_4PFDW2LFVt05iSKquIr16QpnhZFmysCvvoCyDVqmBDwBTtamodVDhiTSoL9eDDGwLosjy31OHUVaGVVDzjE8s3vaic_wZlptbWV9b84CzmzTi09gdRU71OlYzf8dH77n0VvGmdjd2AAB2RLLsl4FrLD4CFKQ3sZ6d6enTlgEstmA0FbPRUsKum8XdU_K4zckV3EeZpNjVwzEM6uKnYotmUuqUW-Zcw9lFEk9L3vwyJW7dzgVsalYkDFRBOh1Ejakpt62kws9rwG-Ksnf2uOaGkC6mt12dumyatalqH8I0scSANfZ4mJYMBTzATVCngpKCkRhqiKeAr6PDbryjArT6_6j3OnpIal-HvSu9rDiGFLswKpcYuNj1372Ki8itbcPWq-dg2iE_go9vuQHtySGq2CZ2Ga5EekWsY6sn73BX9iNpQpCn8bbXcBgbswFcJmr9RU_uDLWZji1R19Juxdw-9hCxKdpLEuPuBcbtHQqbYczWmqrP5c8n1vI4xneAFEPDYhTQC2rBWWeUaYwZf6iMJLOUy3RQ1Mv0-d8oT3hPnUQpBcx4v-bp_c7t3bt4q0oOIK283CEyfWKvMRi1G5rbHAjPcAoz5kwp_Q86LSDG07tKQkuUnClzGz87reYmJeyy6NQwjCUDUg-YL5NfB-HA0ytu-iDpJE-S6a69e-lcHgsDR7weEeqTr1Is0y34Lz0VxkOzGVz6PjtFzj6f1bLQjh1mLCjiA6zzIz5k7cBqaB7pXdo0EmoYmi71NX2Vwqci-H-QclRBs2YOVhr0vDM3VXP_BZ6y4l-7zE1LROgNvgALAYnQLJTpQqNY8MOzPTkvYMtoj7gv13S78m3PBeQy5J5Dsidq3vD5ba246xZDl1hebynCKW1q0cg&uuid=94e3cd18-1ad1-4cd2-9d17-ba9c0d8552ba&authuser=0&nonce=9312v3mei38m8&user=05847473060504281705&hash=sn8mc8pa0u6euhn6o8phag1621bqco2f